# 6. Serving
 --------------------------------------------------------------------

Deploy the model created in the previous notebook and predict the output of the data coming from `serving-stream`. The model execution and results are tracked using MLRun.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

In [1]:
%run config.py

## Load project

In [2]:
from os import path, getenv
from mlrun import load_project, new_project, mount_v3io
import nuclio

In [3]:
project_name = '-'.join(filter(None, [PROJECT_NAME, getenv('V3IO_USERNAME', None)]))
project_path = path.abspath('conf')

if path.isfile(path.join(project_path, 'project.yaml')):
    project = load_project(name=project_name, context=project_path, init_git=False)
else:
    project = new_project(project_name, project_path, init_git=True)

print(f'Project path: {project_path}\nProject name: {project_name}')

Project path: /User/rapid-streaming-ml/conf
Project name: model-deployment-with-streaming-michaelk


### Set the Model

In [4]:
MODEL_NAME = 'my_model'
MODEL_PATH = path.join('/User/examples', PROJECT_NAME, 'model.pkl')
print(f'Set model path:  {MODEL_PATH}')

Set model path:  /User/examples/model-deployment-with-streaming/model.pkl


### Add Serving Functions from the MLRun Functions Marketplace <a id="gs-ml-pipeline-add-functions"></a>


In [5]:
project.set_function('hub://model_server:development', 'serving')

input_stream = f'http://v3io-webapi:8081/users/{getenv("V3IO_USERNAME")}/examples/rapid-prototype/serving-stream@modelserver'
partitions = list(range(0,8))

serving = project.func('serving').apply(mount_v3io())
serving.add_model(MODEL_NAME, MODEL_PATH)
serving.set_envs({'INFERENCE_STREAM' : path.join('users', STREAM_CONFIGS['inference-stream']['path']) })

serving.add_trigger('serving_stream', nuclio.triggers.V3IOStreamTrigger(url=input_stream, partitions=partitions))
serving.spec.config.pop('spec.triggers.http')

serving.spec.min_replicas = 1

In [6]:
serving_addr = serving.deploy()

> 2020-08-06 13:09:35,432 [info] deploy started
[nuclio] 2020-08-06 13:09:37,561 (info) Build complete
[nuclio] 2020-08-06 13:09:49,713 (info) Function deploy complete
[nuclio] 2020-08-06 13:09:49,721 done updating model-deployment-with-streaming-michaelk-sklearn-server, function address: 192.168.226.12:31932


### Trigger the seving func

In [7]:
import requests
import json

feat = [7,799,47,17,143,1560,7,164,11,810.0,162.0,5.0,782.0]
json = json.dumps({'instances': np.array(feat).reshape(1,-1).tolist()})

resp = requests.post(url=f'{serving_addr}/{MODEL_NAME}/predict', 
                     json = json)
print(resp.status_code)
print(resp.text)

200
[0]


### Save Project

In [8]:
project.save(path.join(project_path, 'project.yaml'))